In [1]:
from CGCNN_MT.datamodule.prepare_data import prepare_data
from CGCNN_MT.datamodule.prepare_data import make_prepared_data, get_logger
from pathlib import Path
from tqdm import tqdm
import multiprocessing as mp
import shutil
from functools import partial
import pandas as pd
import json
import warnings
from sklearn.model_selection import train_test_split
import numpy as np
import shutil
warnings.filterwarnings("ignore", category=UserWarning, module="pymatgen.io.cif")
warnings.filterwarnings("ignore", category=UserWarning, module="moftransformer.utils.prepare_data")
warnings.filterwarnings("ignore", category=UserWarning, module="ase.io.cif")

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [5]:
src_csv = "./raw_data/WS24v2/features/features_and_labels.csv"
src_cif_dir1 = Path("./raw_data/WS24v2/data_sets/WS14s/CIFs")
src_cif_dir2 = Path("./raw_data/WS24v2/data_sets/WS24s/CIFs")
tgt_root_dir = Path("./CGCNN_MT/data/WS24")

tgt_root_dir.mkdir(exist_ok=True, parents=True)

In [6]:
print(len(list(src_cif_dir1.glob('*.cif'))))
print(len(list(src_cif_dir2.glob('*.cif'))))

181
911


In [7]:
in_df = pd.read_csv(src_csv)
in_df.head(2)

,MOF_name,data_set,water_label,acid_label,base_label,boiling_label,Di,Df,Dif,VSA,...,f-lig-T-2,f-lig-T-3,f-lig-Z-0,f-lig-Z-1,f-lig-Z-2,f-lig-Z-3,f-lig-chi-0,f-lig-chi-1,f-lig-chi-2,f-lig-chi-3
0,ACOFUU,WS24s,2,0,0,0,7.99458,6.67411,7.99458,2280.10,...,1696.0,2024.0,2032.0,4616.0,6536.0,7248.0,540.5536,1176.932,1966.5344,2565.1968
1,ACOGAB,WS24s,2,0,0,0,5.20086,3.89327,4.83563,1877.51,...,1696.0,2024.0,2032.0,4616.0,6536.0,7248.0,540.5536,1176.932,1966.5344,2565.1968


## Move cif files

In [8]:
mofs_withot_file = []
tgt_cif_dir = tgt_root_dir/"cifs"
# if tgt_cif_dir.exists():
#     shutil.rmtree(tgt_cif_dir)
tgt_cif_dir.mkdir(exist_ok=True)
for i in range(len(in_df)):
    n = in_df.iloc[i]["MOF_name"]
    dataset = in_df.iloc[i]["data_set"]
    if dataset == "WS14s":
        cif_file = src_cif_dir1/f"{n}.cif"
    elif dataset == "WS24s":
        cif_file = src_cif_dir2/f"{n}.cif"
    if not cif_file.exists():
        print(f"{cif_file} does not exist.")
        mofs_withot_file.append(n)
        continue
    shutil.copy(cif_file, tgt_cif_dir)

In [10]:
if len(mofs_withot_file) > 0:
    print(f"There are {len(mofs_withot_file)} MOFs without CIF files.")
    with open(tgt_root_dir/"mofs_withot_cif.txt", "w") as f:
        f.write("\n".join(mofs_withot_file))

### Reneragete zeo++ and RACs features for WS24 datasets.

In [ ]:
from CGCNN_MT.datamodule import clean_cif
import subprocess
import os

## clean cifs
for task in  ["WS24"]:
    task_dir =tgt_root_dir/task
    cif_dir = task_dir/"cifs"
    output_dir = task_dir/"clean_cifs"
    clean_cif.main(cif_dir, output_dir, log_file=task_dir/"clean_cif.log", santize=True, n_cpus=1)

## generate features
for task in  ["WS24"]:
    task_dir =tgt_root_dir/task
    clean_cif_dir = task_dir/"clean_cifs"
    output_dir = task_dir/"features"
    work_dir = Path("./ML/featuring")
    process = subprocess.Popen(
        f"python {work_dir/'feature_generation.py'} --cif_dir {clean_cif_dir} --prob_radius 1.86",
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        shell=True,
        env=os.environ.copy(),
        cwd=str(work_dir)
    )

## Prepare WS24 dataset for CGCNN_MT

In [12]:
import pickle
import os
root_dir = Path("./CGCNN_MT/data/WS24")
cif_dir = root_dir/"clean_cifs"
logger = get_logger(filename=str(root_dir/f"prepare_data_{root_dir.name}.log"))

# Generate prepared data for WS24 dataset
failed_records = []
for cif_file in tqdm(list(cif_dir.glob("*.cif"))):
    g_file_name = cif_file.stem + ".graphdata"
    if (cif_dir/g_file_name).exists():
        with open(cif_dir/g_file_name, 'rb') as f:
            cif_id, atom_num, nbr_idx, nbr_dist, *_, cell_params = pickle.load(f)
        if nbr_idx.shape[0] / atom_num.shape[0] != 10.0:
            print(f"Number of neighbors is not 10 for {g_file_name}")
            os.remove(str(cif_dir/g_file_name))
            flag = make_prepared_data(cif_file, cif_dir, logger=logger, max_num_nbr=10, radius=8)
            if not flag:
                # print(f"Failed to generate graph data for {cif_file}")
                failed_records.append(cif_file.stem)
    else:
        flag = make_prepared_data(cif_file, cif_dir, logger=logger, max_num_nbr=10, radius=8)
        if not flag:
            # print(f"Failed to generate graph data for {cif_file}")
            failed_records.append(cif_file.stem)
            continue
print("Failed records:", len(failed_records))

In [5]:
if len(failed_records) > 0:
    with open(tgt_root_dir/"mofs_prepare_failed.txt", "w") as f:
        f.write("\n".join(failed_records))

In [13]:
print(len(list(cif_dir.glob('*.cif'))))
print(len(list(cif_dir.glob('*.graphdata'))))

1092
1092


## Split the WS24 dataset into training, validation, and test sets

In [27]:
failed_records = []
in_df = pd.read_csv(src_csv)
in_df.insert(3, "water4_label", in_df["water_label"].astype(int))
in_df["water_label"] = in_df["water4_label"].apply(lambda x: 1 if x > 2 else 0)
in_df = in_df[~in_df['MOF_name'].isin(failed_records)]
in_df.rename(columns={"MOF_name": "MofName"}, inplace=True)
print(in_df.shape)
in_df.head(2)

(1092, 166)


,MofName,data_set,water_label,water4_label,acid_label,base_label,boiling_label,Di,Df,Dif,...,f-lig-T-2,f-lig-T-3,f-lig-Z-0,f-lig-Z-1,f-lig-Z-2,f-lig-Z-3,f-lig-chi-0,f-lig-chi-1,f-lig-chi-2,f-lig-chi-3
0,ACOFUU,WS24s,0,2,0,0,0,7.99458,6.67411,7.99458,...,1696.0,2024.0,2032.0,4616.0,6536.0,7248.0,540.5536,1176.932,1966.5344,2565.1968
1,ACOGAB,WS24s,0,2,0,0,0,5.20086,3.89327,4.83563,...,1696.0,2024.0,2032.0,4616.0,6536.0,7248.0,540.5536,1176.932,1966.5344,2565.1968


In [28]:
def split_train_test(df, test_size=0.1, group_column="", random_state=0):
    """Split the dataset into train and test sets."""
    if group_column:
        train_index, test_index = train_test_split(df.index, test_size=test_size, stratify=df[group_column], random_state=random_state)
    else:
        train_index, test_index = train_test_split(df.index, test_size=test_size, random_state=random_state)
    print(f'Train test split successfully: train/test = {len(train_index)}/{len(test_index)}')
    return train_index, test_index


In [29]:
for lb in in_df["water4_label"].unique():
    print(lb, len(in_df[in_df["water4_label"] == lb]))

2 350
4 98
3 555
1 89


In [30]:
random_seed = 42

label_cols = ['water_label', 'acid_label', 'base_label', 'boiling_label']

train_df, test_df = train_test_split(in_df, test_size=0.2, stratify=in_df[label_cols], random_state=random_seed)

train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df[label_cols], random_state=random_seed)

train_df.insert(2, "Partition", "train")
val_df.insert(2, "Partition", "val")
test_df.insert(2, "Partition", "test")

total_df = pd.concat([train_df, val_df, test_df])
ratio_dict = {}
for col in label_cols + ["water4_label"]:
    for lb in total_df[col].unique():
        split_ratios = []
        for split in ["train", "val", "test"]:
            sub_df = total_df.loc[(total_df[col] == lb) & (total_df["Partition"] == split)]
            sub_ratio = len(sub_df) / len(total_df.loc[(total_df[col] == lb)])
            split_ratios.append([sub_ratio, len(sub_df)])
        ratio_dict[f"{col}_{lb}(train/val/test)"] = ' : '.join([f"{c}({r:.2f})" for r, c in split_ratios])
for k, v in ratio_dict.items():
    print(f"{k}: {v}")


water_label_1(train/val/test): 417(0.64) : 105(0.16) : 131(0.20)
water_label_0(train/val/test): 281(0.64) : 70(0.16) : 88(0.20)
acid_label_0(train/val/test): 642(0.64) : 161(0.16) : 201(0.20)
acid_label_1(train/val/test): 56(0.64) : 14(0.16) : 18(0.20)
base_label_0(train/val/test): 658(0.64) : 165(0.16) : 207(0.20)
base_label_1(train/val/test): 40(0.65) : 10(0.16) : 12(0.19)
boiling_label_0(train/val/test): 657(0.64) : 163(0.16) : 206(0.20)
boiling_label_1(train/val/test): 41(0.62) : 12(0.18) : 13(0.20)
water4_label_3(train/val/test): 351(0.63) : 88(0.16) : 116(0.21)
water4_label_2(train/val/test): 220(0.63) : 59(0.17) : 71(0.20)
water4_label_4(train/val/test): 66(0.67) : 17(0.17) : 15(0.15)
water4_label_1(train/val/test): 61(0.69) : 11(0.12) : 17(0.19)


In [31]:
total_df.head()

,MofName,data_set,Partition,water_label,water4_label,acid_label,base_label,boiling_label,Di,Df,...,f-lig-T-2,f-lig-T-3,f-lig-Z-0,f-lig-Z-1,f-lig-Z-2,f-lig-Z-3,f-lig-chi-0,f-lig-chi-1,f-lig-chi-2,f-lig-chi-3
364,KINKUP,WS24s,train,1,3,0,0,0,3.75964,1.31922,...,575.428571,555.142857,816.285714,1839.142857,2564.571429,2361.714286,200.095571,434.170857,681.305829,785.542829
709,TIZYIM,WS24s,train,1,3,0,0,0,4.24601,3.52063,...,3086.000000,2388.000000,817.000000,2248.000000,3380.000000,3306.000000,246.511900,688.021000,1509.685400,1817.937000
886,YUKBOX,WS24s,train,0,2,0,0,0,16.40548,6.25440,...,1646.571429,2096.142857,1518.000000,3450.857143,5413.714286,6724.285714,423.364586,911.453786,1621.134086,2192.630786
477,NABNIO,WS24s,train,1,3,0,0,0,4.17211,3.43410,...,429.000000,455.000000,646.000000,1322.000000,1837.000000,1806.000000,159.829900,323.130500,509.890300,615.175500
986,Ni_HPTZ_2,WS14s,train,1,3,1,0,1,5.28442,4.83200,...,414.000000,417.000000,859.500000,1946.000000,2409.000000,2106.000000,156.341700,320.427800,519.717000,569.667000


In [32]:
total_df.to_csv(tgt_root_dir/'id_prop_feat.csv', index=False)

### Reneragete zeo++ and RACs features for WS24 datasets.

In [32]:
df_new = pd.read_csv('./CGCNN_MT/data/WS24/RAC_and_zeo_features.csv')
df = pd.read_csv("./CGCNN_MT/data/WS24/id_prop_feat.csv")
# df.insert(3, "water4_label", df["water_label"])
# df["water_label"] = df["water4_label"].apply(lambda x: 0 if x<=2 else 1)
print(df_new.shape, df.shape)
df_new.drop(columns=['cif_file'], inplace=True)
df_new.rename(columns={'name': 'MofName'}, inplace=True)
df.rename(columns={'MOF_name': 'MofName', "split": "Partition"}, inplace=True)

id_prop_cols = ["MofName", "Partition", "water_label", "water4_label", "acid_label", "base_label", "boiling_label"]
df_new = df[id_prop_cols].merge(df_new, on="MofName", how="left")

df_new.to_csv("./CGCNN_MT/data/WS24/RAC_and_zeo_features_with_id_prop.csv", index=False)

(1092, 192) (1092, 167)
